In [1]:
import os
import torch

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# PyTorch的数据读取机制

<img style="float: center;" src="images/6.png" width="90%">

## DataLoader

创建可迭代的数据装载器，训练的时候，每一个for循环，每一次iteration，就是从DataLoader中获取一个batch_size大小的数据

torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)

- dataset：Dataset类，决定数据从哪读取以及如何读取
- batch_size：批大小
- num_workers：多进程读取数
- shuffle：每个epoch是否乱序
- drop_last：样本数无法被batch_size整除时，是否舍弃最后一批数据

训练概念：
- epoch：所有训练样本都已输入到模型中，称为一个epoch
- iteration：一批样本输入到模型中，称为一个iteration
- batch_size：批大小，决定一个epoch有多少iteration

假设样本总数80，batch_size为8，则1 epoch = 10 iteration。

假设样本总数87，batch_size为8：
- 如果drop_last=True，则1 epoch = 10 iteration
- 如果drop_last=False，则 1 epoch = 11 iteration

## Dataset

torch.utils.data.Dataset()：Dataset抽象类，所有自定义的Dataset都需要继承它，并且必须复写\_\_getitem\_\_()类方法

```
class Dataset(object):

    def __getitem__(self, index):
        raise NotImplementedError
    
    def __add__(self, other):
        return ConcatDataset([self, other])
```

\_\_getitem\_\_()：Dataset的核心，接收一个索引，返回一个样本。需要编写如何根据这个索引取读取数据部分。

## 数据读取机制实践

1. 读取哪些数据？每一次迭代要读取batch_size大小的样本，那么读哪些样本呢？
2. 从哪里读数据？在硬盘中该如何找数据，在哪设置参数？
3. 怎么读数据？

### 人民币二分类

<img style="float: center;" src="images/7.png" width="70%">

1块的图片100张，100块的图片100张。

任务是训练一个模型，对这两类图片进行分类。

In [2]:
class RMBDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        rmb面额分类任务的Dataset
        :param data_dir: str, 数据集所在路径
        :param transform: torch.transform，数据预处理
        """
        self.label_name = {"1": 0, "100": 1}
        # data_info存储所有图片路径和标签，在DataLoader中通过index读取样本
        self.data_info = self.get_img_info(data_dir)
        self.transform = transform

    # 拿到训练样本
    def __getitem__(self, index):
        path_img, label = self.data_info[index]  # 给定index获取样本
        img = Image.open(path_img).convert('RGB')  # 找到图片转成RGB值

        # 做图片的数据预处理
        if self.transform is not None:
            img = self.transform(img)   # 在这里做transform，转为tensor等等

        # 返回图片的张量形式和标签
        return img, label

    # 一共有多少个样本
    # 不然机器没法根据batch_size的个数去确定有几个iteration
    def __len__(self):
        return len(self.data_info)

    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                img_names = os.listdir(os.path.join(root, sub_dir))
                img_names = list(filter(lambda x: x.endswith('.jpg'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    label = rmb_label[sub_dir]
                    data_info.append((path_img, int(label)))
        
        # 返回一个list元组，每个元素：[(path_img1, label1),(path_img2, label2)...]
        return data_info

In [3]:
# 数据的路径：从哪里读数据
split_dir = os.path.join('data', 'rmb_split')
train_dir = os.path.join(split_dir, 'train')
valid_dir = os.path.join(split_dir, 'valid')

# transforms模块，进行数据预处理
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

# 构建MyDataset实例
# RMBDataset继承抽象类Dataset，并且重写了__getitem__()方法
# 这个类的目的就是传入数据的路径和预处理部分，之后返回数据
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoader
BATCH_SIZE = 16
# dataset接收参数RMBDataset，返回一个样本的张量和标签
# batch_size：每批样本的个数，本例中每次取16张图片的张量和标签
# shuffle：打乱顺序
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)

# 具体训练时，需要便利train_loader，每次取一批数据进行训练
# print(train_loader)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
MAX_EPOCH = 100
# 全部样本循环迭代epoch次
for epoch in range(MAX_EPOCH):
    loss_mean = 0.
    correct = 0.
    total = 0.

    net.train()
    
    # 批次循环，每个epoch中都是一批一批训练
    for i, data in enumerate(train_loader):

        # forward
        inputs, labels = data
        outputs = net(inputs)

        # Compute loss
        optimizer.zero_grad()
        loss = criterion(outputs, labels)

        # backward
        loss.backward()

        # updata weights
        optimizer.step()

for i, data in enumerate(train_loader)取数据过程

1. 程序第一步跳到DataLoader的\_\_iter\_\_(self)方法，先判断是多进程还是单进程读取数据。
<img style="float: center;" src="images/8.png" width="70%">
2. 进入\_\_next\_\_(self)方法，获取下一个样本与标签
  - self.\_\_next\_\_index()：获取下一个样本的index
  - self.dataset_fetcher.fetch(index)：根据index获取下一个样本
<img style="float: center;" src="images/9.png" width="70%">
3. 返回next(self.sampler_iter)
<img style="float: center;" src="images/10.png" width="70%">
4. 进入sampler.py，进入\_\_iter\_\_(self)，一次次采样数据的索引直到够了一个batch_size返回
<img style="float: center;" src="images/12.png" width="70%">
5. 取到index，之前batch_size设置为16，因此通过sampler.py获取16个样本的索引
<img style="float: center;" src="images/11.png" width="70%">
6. 这样获取了一个批次的index，之后根据index取数据。因此第二行代码：
  - data=self.dataset_fetcher.fetch(index)就是取数据去了
7. 进入fetch.py，核心是fetch方法，调用self.dataset[idx]获取数据
<img style="float: center;" src="images/13.png" width="70%">
8. 之后调用RMBDataset中的\_\_getitem\_\_(self, index)方法，获取样本，拿到样本的张量和标签， fetch方法是列表推导式，通过该方法能够获取一个batch大小的样本
<img style="float: center;" src="images/14.png" width="70%">
9. 取完一个批次，进入self.collate_fn(data)进行整合，得到一个批次的data
<img style="float: center;" src="images/15.png" width="70%">
10. 之后可以看到第一个批次的数据样本，train_loader把样本分成一个个batch，通过enumerate进行迭代就可以一批批地获取，然后训练模型，当所有的批次数据都遍历后，完成一次epoch
<img style="float: center;" src="images/16.png" width="70%">

三个问题：
1. 读哪些数据？根据Sampler输出的index决定的
2. 从哪读数据？Dataset的data_dir设置数据的路径然后去读
3. 怎么读数据？Dataset的getitem方法，帮助我们获取一个样本

流程图：
<img style="float: center;" src="images/17.png" width="70%">

# 图像预处理transforms

transforms是常用的图像预处理方法，在torchvision计算机视觉工具包中。

torchvision包主要有三个模块：
- torchvision.transforms：常用的图像预处理方法，比如：
  - 数据标准化、中心化
  - 缩放、裁剪、旋转、翻转、填充
  - 噪声添加、灰度变换、线性变换、仿射变换
  - 亮度、饱和度、对比度
- torchvision.datasets：常用的数据集的dataset实现，MNIST，CIFAR-10，ImageNet等
- torchvision.models：常用的模型预训练，AlexNet，VGG，ResNet，GoogLeNet等

## 二分类任务中的transforms方法

- transforms.Compose：将一系列的transforms方法进行有序的组合包装

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

- transforms.Compose：将一系列的transforms方法进行有序的组合包装，具体实现时依次用包装的方法对图像进行操作。
- transforms.Resize：改变图像大小
- transforms.RandomCrop：对图像进行裁剪
- transforms.ToTensor：将图像传换成张量，同时会进行归一化，将张量的值从0-255转到0-1
- transforms.Normalize：将数据进行标准化

流程图：
<img style="float: center;" src="images/18.png" width="70%">

## 数据标准化

transforms.Normalize：逐channel的对图像进行标准化，$output=(input-mean)/std$

transforms.Normalize(mean, std, inplace=False)

流程图：
1. 进入self.transform(img)函数。
<img style="float: center;" src="images/19.png" width="70%">
2. 进入transform.py，里面的\_\_call\_\_就是那一系列的数据处理方法
<img style="float: center;" src="images/20.png" width="70%">
3. 进入Normalize类，里面有一个call函数调用了PyTorch库里的Normalize函数。Normalize处理有利于加快模型的收敛速度。
<img style="float: center;" src="images/21.png" width="70%">


## transforms的其他图像增强方法

### 数据增强

又称数据增广/数据扩增，对数据集进行变换，使训练集更丰富，从而让模型更具泛化能力。

例如，五年高考三年模拟
<img style="float: center;" src="images/22.png" width="70%">

### 图像裁剪

- transforms.CenterCrop(size)：图像中心裁剪图片，size就是所需裁剪的图片尺寸，如果比原始图像大，则会默认填充0
- transforms.RandomCrop(size, padding=None, pad_if_needed=False, fill=0, padding_mode='constant')：
  - 图像随机位置裁剪size尺寸图片
  - padding设置填充大小
    - 为a时，上下左右各填充a个像素；
    - 为（a，b）时，上下填充b，左右填充a；
    - 为（a，b，c，d）时，左上右下分别填充a，b，c，d）
  - pad_if_need：若图像小于设定的size，则填充
  - padding_mode填充模型：
    - constant：像素值由fill设定
    - edge：像素值由图像边缘像素设定
    - reflect：镜像填充
    - symmetric：镜像填充（复制图像的一部分进行填充）
- transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(3/4, 4/3), interpolation)：随机大小，长宽比裁剪图像，
  - scale：表示随机裁剪面积比例
  - ratio：随机长宽比
  - interpolation：插值方法
- FiveCrop，TenCrop：图像的上下左右及中心裁剪出尺寸为size的5张图片，后者在这5张图片的基础上再水平或者垂直镜像得到10张图片。

### 图像翻转和旋转
- RandomHorizontalFlip(p=0.5)，RandomVerticalFlip(p=0.5)：依概率水平或垂直翻转图片，p为翻转概率
- RandomRotation(degrees, resample=False, expand=False, center=None)：随机旋转图片：
  - degrees：旋转角度
  - resample：重采样方法
  - expand：是否扩大图片以保持原图信息

### 图像变换

- transforms.Pad(padding, fill=0, padding_mode='constant'): 对图片边缘进行填充
- transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0):调整亮度、对比度、饱和度和色相：
  - brightness：亮度调节因子
  - contrast：对比度参数
  - saturation：饱和度参数
  - hue：色相因子
- transfor.RandomGrayscale(num_output_channels, p=0.1)：依概率将图片转换为灰度图，第一个参数是通道数（1或3），p是转换为灰度图像概率值
- transforms.RandomAffine(degrees, translate=None, scale=None, shear=None, resample=False, fillcolor=0)：对图像进行仿射变换（二维的线性变换），由5种基本原子变换（旋转，平移，缩放，错切和翻转）构成。
  - degrees：旋转角度
  - translate：平移区间
  - scale：缩放比例
  - fill_color：填充颜色
  - shear：错切
- transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)：对图像进行随机遮挡，有利于模型识别被遮挡的图片，这个是对张量进行操作，所以需要先转成张量才能做。
  - p：概率值
  - scale：遮挡区域面积
  - ratio：遮挡区域长宽比
  - value：遮挡像素
- transforms.Lambda(lambd): 用户自定义的lambda方法，lambd是一个匿名函数。lambda [arg1 [, arg2…argn]]: expression
- .Resize, .ToTensor, .Normalize: 这三个方法上面具体说过。

## transforms的选择操作

- transforms.RandomChoice([transforms1, transforms2, transforms3]): 从一系列transforms方法中随机选一个
- transforms.RandomApply([transforms1, transforms2, transforms3], p=0.5): 依据概率执行一组transforms操作
- transforms.RandomOrder([transforms1, transforms2, transforms3]): 对一组transforms操作打乱顺序

## 自定义transforms

Compose类里调用一系列transforms方法：
<img style="float: center;" src="images/23.png" width="70%">

对Compose里的transforms方法进行遍历，每次去除一个方法进行执行，即，**transforms方法仅接收一个参数，返回一个参数**，上一个transforms的输出正好是下一个transforms的输入，因此数据类型要注意匹配。

自定义transforms的结构：
<img style="float: center;" src="images/24.png" width="70%">

## 数据增强策略原则

**让训练集与测试机更接近**
- 空间位置上：平移
- 色彩上：灰度图，色彩抖动
- 形状：仿射变换
- 上下文场景：遮挡，填充

# 思维导图

<img style="float: center;" src="images/25.png" width="70%">